In [2]:
import tensorflow as tf
import numpy as np

In [3]:
def residual(u,v,u_t,v_t,u_x,u_xx,u_y,u_yy,v_x,v_xx,v_y,v_yy,p_x,p_y,l1, l2):
    
    f_u = u_t + l1*(u*u_x + v*u_y) + p_x - l2* (u_xx + u_yy)
    f_v = v_t + l1*(u*v_x + v*v_y) + p_y - l2* (v_xx + v_yy)

    return f_u, f_v

class PhysicsInformedNN:
    def __init__(self, lb, ub, layers):

        self.lb = lb
        self.ub = ub
                
        self.layers = layers

        self.model = self.initialize_NN(self,layers)

    def initialize_NN(self,layers):
        model = tf.keras.Sequential()
        model.add(tf.keras.Input(layers[0]))
        scaling_layer = tf.keras.layers.Lambda(
            lambda x: 2.0*(x-self.lb)/(self.ub-self.lb)-1.0)
        model.add(scaling_layer)
        num_layers = len(layers)
        for i in range(1,num_layers-2):
            model.add(tf.keras.layers.Dense(layers[i],
                                            activation=tf.keras.activations.get('tanh'),
                                            kernel_initializer='glorot_normal'))
        model.add(tf.keras.layers.Dense(layers[-1]))

        return model
    

    def loss(self, X, X0, u0, p0):
        psi_and_p = self.model(X0)
        u_pred =  psi_and_p[:,0:1]
        p_pred = psi_and_p[:,1:2]

        loss = tf.reduce_mean(tf.square(u0-u_pred))
        loss += tf.reduce_mean(tf.square(p0-p_pred))

        r1,r2 = self.get_residual(self,X)

        phi_ru = tf.reduce_mean(tf.square(r1))
        phi_rv = tf.reduce_mean(tf.square(r2))

        loss += phi_ru 
        loss += phi_rv

        return loss
    
    def get_residual(self,X):
        with tf.GradientTape(persistent=True) as tape:
            x = X[:,0:1]
            y = X[:,1:2]
            t = X[:,2:3]

            tape.watch(x)
            tape.watch(y)
            tape.watch(t)

            psi_and_p = self.model(tf.stack([x[:,0],y[:,0],t[:,0]], axis=1))

            psi = psi_and_p[:,0:1]
            p = psi_and_p[:,1:2]

            u = tape.gradient(psi,x)
            v = tape.gradient(psi,y)

            u_t = tape.gradient(u,t)
            v_t = tape.gradient(v,t)

            u_x = tape.gradient(u,x)
            u_xx = tape.gradient(u_x,x)

            u_y = tape.gradient(u,y)
            u_yy = tape.gradient(u_y,y)

            v_x = tape.gradient(v,x)
            v_xx = tape.gradient(v_x,x)

            v_y = tape.gradient(v,y)
            v_yy = tape.gradient(v_y,y)

            p_x = tape.gradient(p,x)
            p_y = tape.gradient(p,y)

        del tape
        
        l1 = self.lambda1
        l2 = self.lambda2
        f_u, f_y = residual(u,v,u_t,v_t,u_x,u_xx,u_y,u_yy,v_x,v_xx,v_y,v_yy,p_x,p_y,l1, l2)

        return f_u, f_y
    
    def loss_gradient(self,X,X0,u0, p0):

        with tf.GradientTape(persistent=True) as tape:
            tape.watch(self.model.trainable_variables)
            loss = self.loss(self, X, X0, u0, p0)
            g = tape.gradient(loss, self.model.trainable_variables)

        del tape
        return loss, g
    
    

        






